In [13]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize, ToTensor, Compose, Resize, RandomHorizontalFlip
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device =='cuda':
    print("Run on GPU...")
else:
    print("Run on CPU...")

Run on CPU...


In [17]:
#Hyper-parameters
num_epochs = 4
batch_size = 10
learning_rate = 0.001

In [18]:
data_transforms = {
    'train': Compose([
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': Compose([
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dir = 'archive/train'
test_dir = 'archive/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train']) #100 000 pictures
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)



In [19]:

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Define the first Convolutional Block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Define the second Convolutional Block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        
        # Define the Fully Connected layers
        self.fc1 = nn.Linear(32 * 8 * 8, 64)  # 32*8*8 comes from the feature map size after convolutions and pooling
        self.fc2 = nn.Linear(64, 1)
        
        # Activation Functions
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)  # Flatten the tensor for the fully connected layer
        x = self.relu(self.fc1(x))
        x = self.fc2(x) 
        return x


In [20]:
model = ConvNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)


In [21]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad() #reset all weights
        outputs = model(images) 
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
        if (i+1) % 10000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}],Step [{i+1}/{n_total_steps}] Loss: {loss.item():.4f}')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("Finished Training!")


Epoch [1/4],Step [10000/10000] Loss: 0.0944
Epoch [1/4], Loss: 0.0944
Epoch [2/4],Step [10000/10000] Loss: 0.0605
Epoch [2/4], Loss: 0.0605
Epoch [3/4],Step [10000/10000] Loss: 0.0206
Epoch [3/4], Loss: 0.0206
Epoch [4/4],Step [10000/10000] Loss: 0.1771
Epoch [4/4], Loss: 0.1771
Finished Training!


In [22]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.round(torch.sigmoid(outputs.squeeze()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Accuracy of the model on the test images: 92.99 %
